# Импорт библиотек

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Загрузка данных

In [2]:
data = pd.read_csv('data/train.csv')

In [3]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


# Подготовка данных

In [4]:
# приводим признаки pickup_datetime и dropoff_datetime к типу datetime, т.к. изначально они хранятся в виде строки
data[["pickup_datetime", "dropoff_datetime"]] = data[["pickup_datetime", "dropoff_datetime"]].astype('datetime64[ns]')

In [5]:
data["year"] = data.pickup_datetime.dt.year # создаем новые признаки
data["month"] = data.pickup_datetime.dt.month
data["day"] = data.pickup_datetime.dt.day
data["day_of_week"] = data.pickup_datetime.dt.dayofweek
data["pickup_hour"] = data.pickup_datetime.dt.hour

In [6]:
data.store_and_fwd_flag = LabelEncoder().fit_transform(data.store_and_fwd_flag)

In [7]:
data = data.drop(["id", "pickup_datetime", "dropoff_datetime"], axis=1) # удаляем ненужные признаки

In [8]:
data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,year,month,day,day_of_week,pickup_hour
0,2,1,-73.982155,40.767937,-73.964630,40.765602,0,455,2016,3,14,0,17
1,1,1,-73.980415,40.738564,-73.999481,40.731152,0,663,2016,6,12,6,0
2,2,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,2016,1,19,1,11
3,2,1,-74.010040,40.719971,-74.012268,40.706718,0,429,2016,4,6,2,19
4,2,1,-73.973053,40.793209,-73.972923,40.782520,0,435,2016,3,26,5,13


# Разбиение датасета на X и Y

In [9]:
X = data.drop(["trip_duration"], axis=1)
y = data.trip_duration

# Разбиение выборки на тренировочную и тестовую

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Подбор параметров и обучение моделей

###### Подбор параметров – одна из важнейших задач, необходимых для построения моделей. Изменение параметров модели может принципиально повлиять на качество её работы. Перебор этих параметров вручную может занять колоссальное количество времени.   
###### GridSearchCV – мощный инструмент для автоматического подбора параметров для моделей. Он находит наилучшие параметры, путем обычного перебора создавая модель для каждой возможной комбинации параметров. Важно отметить, что такой подход может быть весьма времязатратным.

In [11]:
linear_regression_parametrs = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}

grid = GridSearchCV(LinearRegression(), linear_regression_parametrs, cv=3)
grid.fit(X_train, y_train)

linear_regression = LinearRegression(**grid.best_params_)

In [12]:
ridge_regression_parameters = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'alpha': range(1, 3, 1),
    'max_iter': range(500, 2000, 500)
}
grid = GridSearchCV(Ridge(), ridge_regression_parameters, cv=3)
grid.fit(X_train, y_train)

ridge_regression = Ridge(**grid.best_params_)

In [15]:
lasso_regression_parameters = {
    'normalize': [True, False],
    'max_iter': range(1000, 2000, 500),
    'selection': ["cyclic", "random"]
}
grid = GridSearchCV(Lasso(), lasso_regression_parameters, cv=3)
grid.fit(X_train, y_train)

lasso_regression = Lasso(**grid.best_params_)

In [16]:
for model in [linear_regression, ridge_regression, lasso_regression]:
    model.fit(X_train, y_train)

# Предсказание с помощью обученных моделей и оценка результата

In [17]:
for model, model_name in [(linear_regression, "linear_regression"), (ridge_regression, "ridge_regression"),
                          (lasso_regression, "lasso_regression")]:
    
    prediction = model.predict(X_test)    
    print(f"{model_name} средняя абсолютная ошибка = {mean_absolute_error(prediction, y_test)}")
    print(f"{model_name} средняя квадратическая ошибка = {mean_squared_error(prediction, y_test)}")
    print(f"{model_name} коэффициент детерминации = {r2_score(prediction, y_test)}\n")

linear_regression средняя абсолютная ошибка = 609.3543008424979
linear_regression средняя квадратическая ошибка = 10600729.649461798
linear_regression коэффициент детерминации = -217.02510677419852

ridge_regression средняя абсолютная ошибка = 609.3587933950652
ridge_regression средняя квадратическая ошибка = 10600725.48456291
ridge_regression коэффициент детерминации = -217.37372068466067

lasso_regression средняя абсолютная ошибка = 610.4449924576745
lasso_regression средняя квадратическая ошибка = 10601565.24089795
lasso_regression коэффициент детерминации = -313.5185987928881

